In [1]:
import os, sys
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('.')))
sys.path.append(parent_dir)

In [7]:
import networkx as nx

G = nx.read_edgelist('net_2', create_using=nx.DiGraph(), nodetype=int)


In [6]:
if G.is_directed():
    print('directed')
    

In [3]:
from networks_gen import configurationG, directed_configurationG

edge_degrees_und = [degree for node, degree in undirected_graph.degree()]
in_degrees_dir = [degree for node, degree in directed_graph.in_degree()]
out_degrees_dir = [degree for node, degree in directed_graph.out_degree()]

G = configurationG(edge_degrees_und)
H = directed_configurationG(in_degrees_dir, out_degrees_dir)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from social_network_algorithms.mining.triangles import clustering_coefficient
from social_network_algorithms.mining.diameter import stream_diam

class Analyzer:
    
    def __init__(self, network):
        self.network = network
        
    def get_degree_distribution(self):
        "Returns the plot of the degree distribution with a log-log scale"
        
        N = self.network.number_of_nodes()
        degrees = [self.network.in_degree(n) for n in self.network.nodes()] if self.network.is_directed() else [self.network.degree(n) for n in self.network.nodes()]
        
        N_k = {k: 0 for k in range(max(degrees) + 1)}
        for k in degrees:
            N_k[k] += 1
                
        p_k = {}
        for k in N_k:
            p_k[k] = N_k[k] / N
        
        # Plotting in log-log scale
        plt.figure(figsize=(8, 6))
        plt.loglog(list(p_k.keys()), list(p_k.values()), marker='o', linestyle='None', color='b')
        plt.title('Log-log Degree Distribution')
        plt.xlabel('Degree')
        plt.ylabel('Frequency')
        plt.grid(True)

        return plt
    
    def get_diameter(self):
        return stream_diam(self.network)
    
    def get_clustering_coefficient(self):
        return clustering_coefficient(self.network)

In [ ]:
analyzer1 = Analyzer(G)
plt = analyzer1.get_degree_distribution()
plt.show()

In [ ]:
analyzer1 = Analyzer(H)
plt = analyzer1.get_degree_distribution()
plt.show()